## Define the Functions

In [1]:
f = @(x) x - x^(1/3) - 2;

error: graphics_toolkit: qt toolkit is not available
error: called from
    graphics_toolkit at line 81 column 5



In [2]:
df = @(x) 1 - x^(-2/3)/3;

In [3]:
g = @(p) p^(1/3) + 2;

## Define Root Finding Methods

The iteration process stops when the difference between two successive approximations is less than the given tolerance delta.

In [4]:
delta = 10^(-8);

### The Bisection method

In [5]:
function [c,err,yc] = bisect(f,a,b,delta)

%Input  - f is the function
%	    - a and b are the left and right endpoints
%	    - delta is the tolerance
%Output - c is the zero
%	    - yc = f(c)
% 	    - err is the error estimate for c

%If f is defined as M-file function, use the @ notation
% call [c,err,yc] = bisect(@f,a,b,delta).
%If f is defined as anonymous functions use the
% call [c,err,yc] = bisect(f,a,b,delta).

fprintf('  k    x(k)       f(x(k))      a(k)        b(k)      |x(k)-x(k-1)|      tol\n');

ya = feval(f,a);
yb = feval(f,b);
if ya * yb > 0,  return, end  %the interval [a,b] does not bracket a root

max1 = 1+round((log(b-a)-log(delta))/log(2)); %the number of iterations
c_old = b;
for k = 1:max1
	c = (a+b) / 2;
	yc = feval(f,c);
	if yc == 0
		err = 0;
	elseif sign(yb) == sign(yc)
		err = abs(c_old - c);
        c_old = c;
        b = c;
		yb = yc;
	else
		err = abs(c_old - c);
        c_old = c;
        a = c;
		ya = yc;
	end
    fprintf('%3d  %11.8f %11.8f  %11.8f  %11.8f  %11.8f  %11.6e\n',k,c,yc,a,b,err,delta);     
	if err < delta, break, end
end

c = (a+b)/2;
err = abs(b-a)/2;
yc = feval(f,c);

end %function


### The Fixed-Point Iteration Method

In [6]:
function [p0,err,k,y] = fixedpoint(g,p0,delta,max1)

%Input    - f is the object function 
%            - df is the derivative of f 
%            - x0 is the initial approximation to a zero of f
%	         - delta is the tolerance for p0
%	         - epsilon is the tolerance for the function values y
%	         - max1 is the maximum number of iterations
%Output - x0 is the Newton-Raphson approximation to the zero
%	         - err is the error estimate for p0
%	         - k is the number of iterations
%	         - y is the function value g(p0)

%If f and df are defined as M-file functions use the @ notation
% call [p0,err,k,y] = fixedpoint(@g,p0,delta,max1).
%If f and df are defined as anonymous functions use the
% call [p0,err,k,y] = fixedpoint(g,p0,delta,max1).

fprintf('  k        x(k)           g(x(k))     |x(k) - x(k-1)|   tol\n');
fprintf('%3d   %12.8f    %12.5e   %12.5e    %12.5e \n',0, p0, g(p0), abs(p0 - g(p0)), delta)

for k = 1:max1	
	p1 = g(p0);	
	err = abs(p1-p0);
	p0 = p1;
	y = g(p0);
    fprintf('%3d   %12.8f    %12.5e   %12.5e    %12.5e\n',k,p0,y,err, delta)
	if (err<delta), break, end
end

end %function

### The Newton Method

In [7]:
function [x0,err,k,y] = newton(f,df,x0,delta,max1)

%Input    - f is the object function 
%            - df is the derivative of f 
%            - x0 is the initial approximation to a zero of f
%	         - delta is the tolerance for p0
%	         - epsilon is the tolerance for the function values y
%	         - max1 is the maximum number of iterations
%Output - x0 is the Newton-Raphson approximation to the zero
%	         - err is the error estimate for p0
%	         - k is the number of iterations
%	         - y is the function value f(p0)

%If f and df are defined as M-file functions use the @ notation
% call [x0,err,k,y] = newton(@f,@df,x0,delta,max1).
%If f and df are defined as anonymous functions use the
% call  [x0,err,k,y] = newton(f,df,x0,delta,max1).

fprintf('  k        x(k)           f(x(k))       |x(k) - x(k-1)|   tol\n');
fprintf('%3d    %12.8f   %12.5e     %12.5e     %12.5e \n',0,x0,f(x0),1,delta)

for k = 1:max1	
	x1 = x0 - f(x0) / df(x0);	
	err = abs(x1-x0);
	x0 = x1;
	y = f(x0);
    fprintf('%3d    %12.8f   %12.5e     %12.5e     %12.5e \n',k,x0,y,err,delta )
	if (err<delta), break, end
end

end %function

In [8]:
function [x1,err,k,y]=secant(f,x0,x1,delta,max1)

%Input    - f is the object function 
%            - x0 and x1 are the initial approximations to a zero of f        
%	         - delta is the tolerance for x1
%	         - max1 is the maximum number of iterations
%Output - x1 is the secant method approximation to the zero
%	         - err is the error estimate for x1
%	         - k is the number of iterations
%	         - y is the function value f(x1)

%If f and df are defined as M-file functions use the @ notation
% call [x1,err,k,y] = secant(@f,x0,x1,delta,max1).
%If f and df are defined as anonymous functions use the
% call  [x1,err,k,y] = secant(f,x0,x1,delta,max1).

fprintf('  k      x(k-1)       x(k)         f(x(k))      |x(k) - x(k-1)|   tol\n');
fprintf('%3d   %12.8f   %12.8f   %12.5e     %12.5e     %12.5e \n',0,x0,x1,f(x1),abs(x1-x0),delta)

for k = 1:max1	
	x2 = x1 - f(x1) * (x1 - x0) / (f(x1) - f(x0));
	err = abs(x2 - x1);
	x0 = x1;
	x1 = x2;
	y = f(x1);
    fprintf('%3d  %12.8f  %12.8f  %12.5e    %12.5e    %10.4e \n',k,x0,x1,y,err,delta)
	if (err<delta),break,end
end

end %function

### Apply all four methods

In [9]:
[c,err,yc] = bisect(f,3,4,delta)

  k    x(k)       f(x(k))      a(k)        b(k)      |x(k)-x(k-1)|      tol
  1   3.50000000 -0.01829449   3.50000000   4.00000000   0.50000000  1.000000e-08
  2   3.75000000  0.19638375   3.50000000   3.75000000   0.25000000  1.000000e-08
  3   3.62500000  0.08884159   3.50000000   3.62500000   0.12500000  1.000000e-08
  4   3.56250000  0.03522131   3.50000000   3.56250000   0.06250000  1.000000e-08
  5   3.53125000  0.00845016   3.50000000   3.53125000   0.03125000  1.000000e-08
  6   3.51562500 -0.00492550   3.51562500   3.53125000   0.01562500  1.000000e-08
  7   3.52343750  0.00176150   3.51562500   3.52343750   0.00781250  1.000000e-08
  8   3.51953125 -0.00158221   3.51953125   3.52343750   0.00390625  1.000000e-08
  9   3.52148438  0.00008959   3.51953125   3.52148438   0.00195312  1.000000e-08
 10   3.52050781 -0.00074632   3.52050781   3.52148438   0.00097656  1.000000e-08
 11   3.52099609 -0.00032837   3.52099609   3.52148438   0.00048828  1.000000e-08
 12   3.52124023 -0.00

In [10]:
[x0,err,k,y] = fixedpoint(g,3,delta,50)

  k        x(k)           g(x(k))     |x(k) - x(k-1)|   tol
  0     3.00000000     3.44225e+00    4.42250e-01     1.00000e-08 
  1     3.44224957     3.50990e+00    4.42250e-01     1.00000e-08
  2     3.50989745     3.51972e+00    6.76479e-02     1.00000e-08
  3     3.51972430     3.52114e+00    9.82686e-03     1.00000e-08
  4     3.52114127     3.52135e+00    1.41696e-03     1.00000e-08
  5     3.52134537     3.52137e+00    2.04099e-04     1.00000e-08
  6     3.52137476     3.52138e+00    2.93937e-05     1.00000e-08
  7     3.52137899     3.52138e+00    4.23311e-06     1.00000e-08
  8     3.52137960     3.52138e+00    6.09626e-07     1.00000e-08
  9     3.52137969     3.52138e+00    8.77945e-08     1.00000e-08
 10     3.52137970     3.52138e+00    1.26436e-08     1.00000e-08
 11     3.52137971     3.52138e+00    1.82085e-09     1.00000e-08
x0 =  3.5214
err =  0.0000000018209
k =  11
y =  3.5214


In [11]:
[x0,err,k,y] = newton(f,df,3,delta,50)

  k        x(k)           f(x(k))       |x(k) - x(k-1)|   tol
  0      3.00000000   -4.42250e-01      1.00000e+00      1.00000e-08 
  1      3.52664429    4.50679e-03      5.26644e-01      1.00000e-08 
  2      3.52138015    3.77141e-07      5.26415e-03      1.00000e-08 
  3      3.52137971    2.66454e-15      4.40593e-07      1.00000e-08 
  4      3.52137971    0.00000e+00      3.10862e-15      1.00000e-08 
x0 =  3.5214
err =    3.1086e-15
k =  4
y = 0


In [12]:
[x1,err,k,y]=secant(f,3,3.5,delta,50)

  k      x(k-1)       x(k)         f(x(k))      |x(k) - x(k-1)|   tol
  0     3.00000000     3.50000000   -1.82945e-02      5.00000e-01      1.00000e-08 
  1    3.50000000    3.52157597   1.68001e-04     2.15760e-02    1.0000e-08 
  2    3.52157597    3.52137964  -5.74138e-08     1.96332e-04    1.0000e-08 
  3    3.52137964    3.52137971  -1.79634e-13     6.70730e-08    1.0000e-08 
  4    3.52137971    3.52137971   0.00000e+00     2.10054e-13    1.0000e-08 
x1 =  3.5214
err =    2.1005e-13
k =  4
y = 0
